In [9]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString

In [10]:
#reading in the routes and poi data
routes = pd.read_csv('cleaned_routes_data/cleaned_all_routes_data_long.csv')
poi = pd.read_csv('all_nodes_uncleaned.csv').iloc[0:1000, :] #using smaller sample of uncleaned data for experimentation

In [11]:
#converting routes df into a GeoDataFrame
gdf_routes = gpd.GeoDataFrame(routes, geometry=gpd.points_from_xy(routes.latitude, routes.longitude))
gdf_routes.drop(['Unnamed: 0', 'lat_lgt'], axis=1, inplace=True)
gdf_routes

,route_id,num_of_waypoint,latitude,longitude,geometry
0,1005019,0,52.50607,13.33208,POINT (52.50607 13.33208)
1,1005019,1,52.50553,13.33163,POINT (52.50553 13.33163)
2,1005019,2,52.50525,13.33148,POINT (52.50525 13.33148)
3,1005019,3,52.50515,13.33337,POINT (52.50515 13.33337)
4,1005019,4,52.50520,13.33366,POINT (52.50520 13.33366)
...,...,...,...,...,...
119839,933359,151,52.50444,13.38246,POINT (52.50444 13.38246)
119840,933359,152,52.50525,13.38633,POINT (52.50525 13.38633)
119841,933359,153,52.50643,13.38615,POINT (52.50643 13.38615)
119842,933359,154,52.50648,13.39023,POINT (52.50648 13.39023)


In [25]:
#converting routes data to lines and creating a gdf from it
routes_linestring_series = gdf_routes.groupby(['route_id'])['geometry'].apply(lambda x: LineString(x.tolist()))
gdf_lined_routes = routes_linestring_series.to_frame()
gdf_lined_routes

,geometry
route_id,
113043,"LINESTRING (52.45147 13.69072, 52.45147 13.690..."
113104,"LINESTRING (52.50054 13.44143, 52.50090 13.442..."
113232,"LINESTRING (52.42580 13.62998, 52.42403 13.629..."
128939,"LINESTRING (52.54556 12.99893, 52.54447 13.004..."
148436,"LINESTRING (52.50736 13.52251, 52.50712 13.522..."
...,...
3673084,"LINESTRING (52.53555 13.20110, 52.53553 13.201..."
3674009,"LINESTRING (52.53883 13.21284, 52.53883 13.212..."
3674010,"LINESTRING (52.53883 13.21284, 52.53883 13.212..."


In [26]:
#converting poi df into a GeoDataFrame
gdf_poi = gpd.GeoDataFrame(poi, geometry=gpd.points_from_xy(poi.lat, poi.lon))
gdf_poi

,Unnamed: 0,type,id,lat,lon,tags,geometry
0,0,node,78252154,52.523744,13.398627,"{'addr:city': 'Berlin', 'addr:country': 'DE', ...",POINT (52.52374 13.39863)
1,1,node,87036263,52.532985,13.384282,"{'amenity': 'atm', 'brand': 'Sparda-Bank', 'br...",POINT (52.53299 13.38428)
2,2,node,89275133,52.518025,13.406956,"{'amenity': 'atm', 'name': 'Bankhaus August Le...",POINT (52.51802 13.40696)
3,3,node,213106623,52.542170,13.441137,"{'addr:country': 'DE', 'addr:housenumber': '87...",POINT (52.54217 13.44114)
4,4,node,213113204,52.542750,13.392862,"{'amenity': 'atm', 'name': 'Berliner Sparkasse...",POINT (52.54275 13.39286)
...,...,...,...,...,...,...,...
995,995,node,9052654336,52.479879,13.421411,"{'amenity': 'atm', 'brand': 'Euronet', 'brand:...",POINT (52.47988 13.42141)
996,996,node,9099183859,52.500519,13.441800,"{'amenity': 'atm', 'indoor': 'no', 'operator':...",POINT (52.50052 13.44180)
997,997,node,9144971609,52.544339,13.378543,{'amenity': 'atm'},POINT (52.54434 13.37854)
998,998,node,9165913334,52.571453,13.379997,"{'amenity': 'atm', 'brand': 'Euronet', 'brand:...",POINT (52.57145 13.38000)


In [ ]:
#spacial joining both datasets on nearest
poi_routes = gpd.sjoin_nearest(gdf_lined_routes, gdf_poi, how='left', max_distance=1000, distance_col=True) #not working yet
poi_routes

GeoDataFrame-methods: 
crosses(other[, align]) Returns a Series of dtype('bool') with value True for each aligned geometry that cross other.
distance(other[, align]) Returns a Series containing the distance to aligned other.
intersection(other[, align]) Returns a GeoSeries of the intersection of points in each aligned geometry with other.
intersects(other[, align]) Returns a Series of dtype('bool') with value True for each aligned geometry that intersects other.
join(other[, on, how, lsuffix, rsuffix, sort]) Join columns of another DataFrame.
mad([axis, skipna, level]) Return the mean absolute deviation of the values over the requested axis.
sjoin(df, *args, **kwargs) Spatial join of two GeoDataFrames. 
symmetric_difference(other[, align]) Returns a GeoSeries of the symmetric difference of points in each aligned geometry with other.
sjoin_nearest(right[, how, max_distance, ...]) Spatial join of two GeoDataFrames based on the distance between their geometries.

more: https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html